In [114]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import qgrid 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Créez un dataframe contenant les informations de population de chaque pays. Calculez le nombre total d’humains sur la planète. Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires.

# Question 1 : donnez le résultat de votre calcul pour l'année 2013.

In [115]:
# Télécharger csv population
populations = pd.read_csv("fr_population.csv")
populations.head(5)

# Utilisation grille
qgrid.show_grid (populations, show_toolbar=True)

#Description et détéction valeur à supprimer du Dataset

populations.describe(include='all')
populations.describe(include='O')
populations[populations['Description du Symbole']=='Agrégat, peut inclure des données officielles, semi-officielles, estimées ou calculées']

# Valeur Chine 1416667 * 1000(Unité).
ChineD = populations[populations['Zone']=='Chine']['Valeur']*1000
ChineD

# La somme de tous les pays moins la valeur Chine à supprimer du dataset.
# banque mondiale 7,171 milliard
#https://donnees.banquemondiale.org/indicator/SP.POP.TOTL
totalP = populations['Valeur'].sum()*1000-ChineD
totalP

# Supression de la ligne 33 (Chine).

populations.drop(33,axis=0, inplace=True)
populations

# Affectation 6997326000 dans la variable one.

one = 6997326000
print('La population mondiale en 2013 est de : {one} milliards'.format(one=one))


La population mondiale en 2013 est de : 6997326000 milliards


# Question 2 : Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici :soleil: ). C'est une équation à 3 termes de type \(a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]\) ) faisant intervenir chacune des 11 quantités données ci dessus. Illustrez cette équation avec l'exemple du blé en France. Pour avoir un indice, cliquez sur "Définitions et Standards" sur la page de téléchargement des données.

In [116]:
# Télécharger le csv vegetaux et supprimer la chine agrégat

vegetaux = pd.read_csv("fr_vegetaux.csv")
vegetaux.drop(vegetaux.index[19971: 20831],0, inplace = True)

# Projection(Zone, Produits), Resctriction(France , Blé)
France_ble = vegetaux[(vegetaux['Zone']=='France')&(vegetaux['Produit']=='Blé')]
France_ble
# Chercher les variables dans Élément
France_ble1 = France_ble[France_ble['Élément'].isin(['Production','Importations - Quantité','Disponibilité intérieure',
                                 'Exportations - Quantité', 'Variation de stock', 'Nourriture','Semences', 'Pertes','Traitement',
       'Autres utilisations (non alimentaire)', 'Aliments pour animaux' ])][['Élément', 'Produit', 'Valeur']]

France_ble1

# Table pivot sur dataframe(France_ble1)

result = pd.pivot_table(France_ble1,values='Valeur', index='Produit', columns='Élément', aggfunc=np.sum, fill_value=0)
result1 = result.reset_index().rename_axis(None, axis=1)
result1 = result1.rename(columns={'Disponibilité intérieure':'Disponibilité intérieure[A]'})

# Créations colonnes faisant intervenir chacune des 11 quantités données ci dessus

result1['B']= result1['Production']+result1['Importations - Quantité']-result1['Exportations - Quantité']+result1['Variation de stock']
result1['C']= result1['Nourriture']+result1['Aliments pour animaux']+result1['Pertes']+result1['Traitement']+result1['Autres utilisations (non alimentaire)']+result1['Semences']
result1[['Produit','Disponibilité intérieure[A]','B','C']]

#[A]=Disponibilités intérieures
#[B]=Production + importations - exportations + variations des stocks
#[C]=Nourriture + Aliments pour animaux + Pertes + Traitement + Autres utilisations + Semences

#[A]=[B]=[C]


,Produit,Disponibilité intérieure[A],B,C
0,Blé,20298,20298,20298


# Question 3 : Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.¶
# Vous ferez cela à partir de ces informations :

# Population de chaque pays ; Disponibilité alimentaire donnée pour chaque produit et pour chaque pays en kcal/personne/jour. Disponibilité alimentaire en protéines donnée pour chaque produit et pour chaque pays en g/personne/jour.



In [117]:
# Restriction Disponibilité alimentaire (Kcal/personne/jour)
kcal_vegetaux = vegetaux[vegetaux['Élément']=='Disponibilité alimentaire (Kcal/personne/jour)']

# La table animaux  
# Supprimer la chine 
# Restriction Disponibilité alimentaire (Kcal/personne/jour)

animaux = pd.read_csv("fr_animaux.csv")
animaux.drop(animaux.index[6985:7243],0,inplace=True)
kcal_animal = animaux[animaux['Élément']=='Disponibilité alimentaire (Kcal/personne/jour)']

# Projection 

kcal_vegetaux = kcal_vegetaux[["Zone","Produit","Valeur"]]
kcal_animal = kcal_animal[["Zone","Produit","Valeur"]]

# Concaténation entre kcal_animal et kcal_vegetaux

kcal_pr = pd.concat([kcal_animal, kcal_vegetaux], axis=0).reset_index()[["Zone","Produit","Valeur"]]
kcal_pr.head()

# Projections et renommer colone sur la table populations

populations = populations[["Zone","Valeur"]]
populations = populations.rename(columns={"Valeur":"Population"})
populations.head()

# Merge entre la kcal produit et populations

kcal_prod = kcal_pr.merge(populations, on="Zone", how='left')
kcal_prod.head()

#Création colonne disponibilité alimentaire kcal 
# Pour chaque pays et produits

kcal_prod['Dispo_alim_kcal'] = kcal_prod["Valeur"]*kcal_prod["Population"]*365*1000
kcal_prod.head()

############ La disponibilté alimentaire kcal pour chaque pays ########################

dispo_kcal_pays = kcal_prod[["Zone","Dispo_alim_kcal"]].groupby("Zone").sum().reset_index()
dispo_kcal_pays.head()

,Zone,Dispo_alim_kcal
0,Afghanistan,2.327314e+13
1,Afrique du Sud,5.817498e+13
2,Albanie,3.692166e+12
3,Algérie,4.712586e+13
4,Allemagne,1.057743e+14


In [118]:
################ La disponibilité alimentaire kcal pour chaque produits #############

dispo_kcal_produit = kcal_prod[["Produit","Dispo_alim_kcal"]].groupby("Produit").sum().reset_index()
dispo_kcal_produit.head()


,Produit,Dispo_alim_kcal
0,Abats Comestible,1.723963e+13
1,"Agrumes, Autres",2.473770e+12
2,Aliments pour enfants,2.795015e+12
3,Ananas,7.077457e+12
4,Animaux Aquatiques Autre,5.511938e+11


#### Disponibilité alimentaire en protéines donnée pour chaque produit et pour chaque pays en g/personne/jour.

In [119]:
# Restriction 'Disponibilité de protéines en quantité (g/personne/jour)'
# Table animaux et vegetaux 

proteine_vegetaux = vegetaux[vegetaux['Élément']=='Disponibilité de protéines en quantité (g/personne/jour)']
proteine_animaux = animaux[animaux['Élément']=='Disponibilité de protéines en quantité (g/personne/jour)']

# Projection sur table animaux et vegetaux pour les proteines 
# Zone, Produit et Valeur 

proteine_animaux = proteine_animaux[['Zone', 'Produit', 'Valeur']]
proteine_vegetaux = proteine_vegetaux[['Zone', 'Produit', 'Valeur']]

# Concaténation entre la proteine_animaux et proteine_vegetaux 

Proteine_pr = pd.concat([proteine_animaux, proteine_vegetaux], axis=0).reset_index()[['Zone', 'Produit', 'Valeur']]
Proteine_pr.head()

# Merge entre les proteines produits et la populations

Proteine_prod = Proteine_pr.merge(populations, on='Zone', how='left')

#Création colonne disponibilité alimentaire kg de proteine pour chaque pays et produits

Proteine_prod['Dispo_alim_kg_Proteines']=(Proteine_prod['Valeur']/1000)*(Proteine_prod['Population']*1000)*365
Proteine_prod.head()

############La disponibilité alimentaire en kg de proteines pour chaque pays ################
# Groupby pays aggrégation sum
dispo_alim_pays_prot = Proteine_prod[['Zone','Dispo_alim_kg_Proteines']].groupby('Zone').sum().reset_index()
dispo_alim_pays_prot.head()

,Zone,Dispo_alim_kg_Proteines
0,Afghanistan,6.496852e+08
1,Afrique du Sud,1.643732e+09
2,Albanie,1.289826e+08
3,Algérie,1.315460e+09
4,Allemagne,3.061507e+09


In [120]:
############La disponibilité alimentaire en kg de proteines pour chaque produits ################
# Groupby produits aggrégation sum

dispo_alim_prod_prot = Proteine_prod[['Produit','Dispo_alim_kg_Proteines']].groupby('Produit').sum().reset_index()
dispo_alim_prod_prot.head()

,Produit,Dispo_alim_kg_Proteines
0,Abats Comestible,2.799873e+09
1,"Agrumes, Autres",5.205833e+07
2,Aliments pour enfants,1.181439e+08
3,Ananas,5.490385e+07
4,Animaux Aquatiques Autre,4.588584e+07


# Question 4 : A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg. Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

In [121]:
#Restriction Nourriture pour table vegetaux 
#Restriction Nourriture pour table animaux 

qt_vege = vegetaux[vegetaux['Élément']=='Nourriture']
qt_anim = animaux[animaux['Élément']=='Nourriture']

# Projection zone produit et valeur pour la table vegetaux 
# Projection zone produit valeur pour la table animaux 

qt_vege = qt_vege[['Zone', 'Produit', 'Valeur' ]]
qt_anim = qt_anim[['Zone', 'Produit', 'Valeur' ]]

#Concaténation entre qt_anim et qt_vege

qt_pr = pd.concat([qt_anim, qt_vege], axis=0).reset_index()[['Zone', 'Produit', 'Valeur' ]]
qt_pr = qt_pr.rename(columns={'Valeur':'Dispo_1000T'})
qt_pr.head(2)
kcal_prod.head(2)

# Merge entre kcal_prod et qt_pr

kcal_prod1 = kcal_prod.merge(qt_pr, on=['Zone', 'Produit'], how='left')

# Group by produit avec aggrégation sum 
# Projections Produit , Dispo_alim_kca et Dispo_1000T

kcal_prod2 = kcal_prod1[['Produit', 'Dispo_alim_kcal','Dispo_1000T']].groupby("Produit").sum().reset_index()


# Création colonne Ratio énergie/poids pour chaques produits

kcal_prod2['Ratio'] = kcal_prod2['Dispo_alim_kcal']/(kcal_prod2['Dispo_1000T']*1000000)

# Le ratio energie/poids pour oeufs en kg

kcal_prod2[kcal_prod2['Produit']=='Oeufs']


,Produit,Dispo_alim_kcal,Dispo_1000T,Ratio
57,Oeufs,9.193986e+13,64331.0,1429.168896


# En suivant la même méthodologie, calculez également le pourcentage de protéines de chaque produit (pour chaque pays). Ce pourcentage est obtenu en calculant le ratio "poids de protéines/poids total" (attention aux unités utilisées). Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la teneur en protéines de l'avoine.

In [122]:
# Merge entre la table proteine_prod et la qt_pr

Proteine_prod1 = Proteine_prod.merge(qt_pr, on=['Zone', 'Produit'], how='left')

# Group by Produit avec aggrégation sum
# Projection 'Produit', 'Dispo_alim_kg_Proteines','Dispo_1000T'

Proteine_prod2 = Proteine_prod1[['Produit', 'Dispo_alim_kg_Proteines','Dispo_1000T']] .groupby('Produit').sum().reset_index()

#Création colonne Ratio_protéines/poids total

Proteine_prod2['Ratio_protéines/poids total'] = Proteine_prod2['Dispo_alim_kg_Proteines']/(Proteine_prod2['Dispo_1000T']*1000000)

# Création colonne pourcentage de proteine 

Proteine_prod2['%proteine'] = Proteine_prod2['Ratio_protéines/poids total']*100
Proteine_prod2

,Produit,Dispo_alim_kg_Proteines,Dispo_1000T,Ratio_protéines/poids total,%proteine
0,Abats Comestible,2.799873e+09,15669.0,1.786887e-01,17.868869
1,"Agrumes, Autres",5.205833e+07,11352.0,4.585829e-03,0.458583
2,Aliments pour enfants,1.181439e+08,761.0,1.552482e-01,15.524822
3,Ananas,5.490385e+07,20754.0,2.645459e-03,0.264546
4,Animaux Aquatiques Autre,4.588584e+07,1249.0,3.673806e-02,3.673806
...,...,...,...,...,...
91,Viande de Suides,1.186113e+10,112100.0,1.058085e-01,10.580849
92,Viande de Volailles,1.317308e+10,104871.0,1.256122e-01,12.561222
93,"Viande, Autre",1.209520e+09,6823.0,1.772710e-01,17.727099
94,Vin,1.934500e+03,20938.0,9.239182e-08,0.000009


# Question 5 : Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.
# Étonnamment, il arrive que ce ratio soit différent en fonction du pays. Il faudra donc réaliser pour chaque aliment une moyenne sur les différents pays. Vous créerez donc une nouvelle table grâce à une agrégation. Attention à bien retirer les valeurs égales à 0 afin de ne pas fausser le calcul de la moyenne.
# Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [124]:
# Restriction Disponibilité alimentaire (Kcal/personne/jour)et Valeur différent de 0
# Pour les tables vegetaux et animaux

kcal_veg0 = vegetaux[(vegetaux['Élément']=='Disponibilité alimentaire (Kcal/personne/jour)')
                                                      &(vegetaux['Valeur']!=0)]

kcal_anim0 = animaux[(animaux['Élément']=='Disponibilité alimentaire (Kcal/personne/jour)')
                                                      &(animaux['Valeur']!=0)]

# Projection "Zone","Produit","Valeur" pour tables vegetaux et animaux 
kcal_vege0 = kcal_veg0[["Zone","Produit","Valeur"]]
kcal_anim0 = kcal_anim0[["Zone","Produit","Valeur"]]

# Concaténation entre kcal_anim0 et kcal_vege0 
kcal_pr_0 = pd.concat([kcal_anim0, kcal_vege0 ], axis=0).reset_index()[["Zone","Produit","Valeur"]]

# Merge entre kcal_pr_0 et la populations

kcal_pr_pays_0=kcal_pr_0.merge(populations, on='Zone', how='left')

# Création colonne Dispo_alim0_kcal

kcal_pr_pays_0['Dispo_alim0_kcal'] = kcal_pr_pays_0["Valeur"]*kcal_pr_pays_0["Population"]*365*1000
kcal_pr_pays_0.head()


# Restriction Nourriture et Valeur différent de 0
# Pour les tables vegetaux et animaux

qt0_vege = vegetaux[(vegetaux['Élément']=='Nourriture')
                                                      &(vegetaux['Valeur']!=0)]

qt0_anim = animaux[(animaux['Élément']=='Nourriture')
                                                      &(animaux['Valeur']!=0)]

# Projections "Zone","Produit","Valeur pour les tables animaux et vegetaux différent de 0
#Nourriture
qt0_vege = qt0_vege[["Zone","Produit","Valeur"]]
qt0_anim = qt0_anim[["Zone","Produit","Valeur"]]

# Concaténation qt0_anim et qt0_vege
qt0_pr = pd.concat([qt0_anim, qt0_vege], axis=0).reset_index()[["Zone","Produit","Valeur"]]

#Renommer colonne 

qt0_pr = qt0_pr.rename(columns={'Valeur' : 'Dispo_alim0_1000T'})

# merge entre kcal_pr_pays_0 et qt0_pr

kcal0_pr= kcal_pr_pays_0.merge(qt0_pr, on=['Zone', 'Produit'], how='left')


# Group by Produit avec aggregation mean 
# Création colone Ratio_kcal/kg et tri décroissant

Ratio_kcal=kcal0_pr[['Produit', 'Dispo_alim0_kcal', 'Dispo_alim0_1000T']].groupby('Produit').mean().reset_index()
Ratio_kcal['Ratio_kcal/kg']=Ratio_kcal['Dispo_alim0_kcal']/(Ratio_kcal['Dispo_alim0_1000T']*1000000)
Ratio_kcal.head()

################### Les 20 aliments les plus caloriques #############################

Ratio_kcal[['Produit', 'Ratio_kcal/kg']].sort_values(by='Ratio_kcal/kg', ascending=False).head(20)

,Produit,Ratio_kcal/kg
40,Huile de Son de Riz,8438.335730
37,Huile de Palme,8173.932495
42,Huile de Tournesol,7815.448979
38,Huile de Palmistes,7754.862730
35,Huile de Colza&Moutarde,7710.686997
31,Huile Graines de Coton,7623.340008
39,Huile de Soja,7377.224008
30,Huil Plantes Oleif Autr,7143.127483
32,Huile d'Arachide,6944.826089
44,Huiles de Poissons,6802.597612


### Pour le pourcentage proteine avec la moyenne 

In [125]:
# Restriction Disponibilité de protéines en quantité (g/personne/jour)et Valeur différent de 0
# Pour les tables vegetaux et animaux

proteine_veg0 = vegetaux[(vegetaux['Élément']=='Disponibilité de protéines en quantité (g/personne/jour)')
                                                      &(vegetaux['Valeur']!=0)]

proteine_anim0 = animaux[(animaux['Élément']=='Disponibilité de protéines en quantité (g/personne/jour)')
                                                      &(animaux['Valeur']!=0)]

# Projection "Zone","Produit","Valeur" pour tables vegetaux et animaux 
proteine_vege0 = proteine_veg0[["Zone","Produit","Valeur"]]
proteine_anim0 = proteine_anim0[["Zone","Produit","Valeur"]]

# Concaténation entre proteine_anim0 et proteine_vege0 
proteine_pr_0 = pd.concat([proteine_anim0, proteine_vege0 ], axis=0).reset_index()[["Zone","Produit","Valeur"]]

# Merge entre proteine_pr_0 et populations

proteine_pr_pays0 = proteine_pr_0.merge(populations, on='Zone', how='left')

# Création colonne Dispo_alim0_kg_Proteines

proteine_pr_pays0['Dispo_alim0_kg_Proteines']=(proteine_pr_pays0['Valeur']/1000)*(proteine_pr_pays0['Population']*1000)*365
proteine_pr_pays0.head()


# Merge entre proteine_pr_pays0 et qt0_pr

proteine_pr_pays0 = proteine_pr_pays0.merge(qt0_pr, on=['Zone', 'Produit'], how='left')

# group By produit avec aggrégation mean 
# Création colonne Ratio_protéines/poids total
# Création '%_proteine'

Ratio_prot = proteine_pr_pays0[['Produit','Dispo_alim0_kg_Proteines','Dispo_alim0_1000T' ]].groupby('Produit').mean().reset_index()
Ratio_prot['Ratio_protéines/poids total'] = Ratio_prot['Dispo_alim0_kg_Proteines']/(Ratio_prot['Dispo_alim0_1000T']*1000000)
Ratio_prot['%_proteine']=Ratio_prot['Ratio_protéines/poids total']*100
Ratio_prot.head()

##################### Les 20 aliments les plus riches en proteines ######################

Ratio_prot[['Produit', '%_proteine']].sort_values(by='%_proteine', ascending=False).head(20)


,Produit,%_proteine
74,Soja,22.568450
5,Arachides Decortiquees,19.247639
29,Haricots,18.367958
63,Pois,17.665770
44,Légumineuses Autres,17.643836
0,Abats Comestible,16.533839
86,"Viande, Autre",14.493855
83,Viande de Bovins,13.615099
62,"Plantes Oleiferes, Autre",13.531722
66,Poissons Pelagiques,13.129789


In [ ]:
#Ratio_kcal1 = Ratio_kcal[['Produit', 'Ratio_kcal/kg']]
# Ratio_prot = Ratio_prot[['Produit', 'Ratio_protéines/poids total']]

# Question 6 : Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [127]:
# Restriction Disponibilité intérieure
# Projections 'Produit', 'Valeur', 'Année

Dispo_Int_vege = vegetaux[vegetaux['Élément']=='Disponibilité intérieure']
Dispo_Int_vege = Dispo_Int_vege[['Produit', 'Valeur', 'Année']].reset_index()[['Produit', 'Valeur','Année']]
Dispo_Int_vege.head()

# Le ratio_kcal1 
Ratio_kcal1 = Ratio_kcal[['Produit', 'Ratio_kcal/kg']]

# Merge entre Dispo_Int_vege et Ratio_kcal1

Dispo_Int_vege1=Dispo_Int_vege.merge(Ratio_kcal1 , on='Produit', how='left')
Dispo_Int_vege1.head(2)


# Création colonne Disponibilité_intérieure(conversion milliers de tonnes en kg) 
# Group by Année

Dispo_Int_vege1['Disponibilité_intérieure']=Dispo_Int_vege1['Valeur']*1000000*Dispo_Int_vege1['Ratio_kcal/kg']
Dispo_Mon_kcal=Dispo_Int_vege1[['Année','Disponibilité_intérieure']].groupby('Année').sum()
Dispo_Mon_kcal = Dispo_Mon_kcal.rename(columns={'Disponibilité_intérieure': 'Disponibilité_intérieure_mondiale_kcal'}).reset_index()
Dispo_Mon_kcal


,Année,Disponibilité_intérieure_mondiale_kcal
0,2013,1.282737e+16


# Question 7 : Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [146]:
# Besoin moyenne nécessaire est de 2 500 kcal/personne/jour
#https://fr.wikipedia.org/wiki/Ration_alimentaire

besoin_kcal_jour = 2500
Disponibilité_int_mond = 1.282737e+16

Disponibilité_int_mond/ 365 / besoin_kcal_jour

# Le nombre d'humains qui pourraient etre nourris 

Disponibilité_int_mond/ 365 / besoin_kcal_jour
Humain_nourris = 14057391780.82
Population_mond=6997326000

print("Le nombre d'humains qui pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture: {Humain_nourris} milliards".format(Humain_nourris=Humain_nourris))

print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

one = round (Humain_nourris / Population_mond,2)
print("Le nombre de fois qu'on pourraient nourrir la population mondiales avec la disponibilité intérieure mondiale de produits végétaux est de : {one} ".format(one=one))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

one = round(Humain_nourris / Population_mond*100,2)
print("Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité intérieure de produits végétaux est de   : {one} %".format(one=one))


Le nombre d'humains qui pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture: 14057391780.82 milliards
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le nombre de fois qu'on pourraient nourrir la population mondiales avec la disponibilité intérieure mondiale de produits végétaux est de : 2.01 
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité intérieure de produits végétaux est de   : 200.9 %


### Disponibilité intérieure mondiale en proteine kg

In [145]:
# Projections 'Produit', 'Ratio_protéines/poids total'
# Merge entre le ratio_prot1 et la disponibilité intérieure vegetaux
# Création colonne Disponibilité intérieure

Ratio_prot1=Ratio_prot[['Produit', 'Ratio_protéines/poids total']]
Dispo_Int_vegeProt=Dispo_Int_vege.merge(Ratio_prot1, on='Produit', how='left')
Dispo_Int_vegeProt['Disponibilité_intérieure'] = Dispo_Int_vegeProt['Valeur']*1000000*Dispo_Int_vegeProt['Ratio_protéines/poids total']
Dispo_Int_vegeProt.head()


# Group by Année 
# renomer colonne 

Dispo_Int_vegeProt1=Dispo_Int_vegeProt[['Année','Disponibilité_intérieure']].groupby('Année').sum().reset_index().rename(columns={'Disponibilité_intérieure':'Disponibilité_intérieure_prot_kg'})
Dispo_Int_vegeProt1

#https://www.cerin.org/rapports/besoin-en-proteines/#:~:text=L'Apport%20Nutritionnel%20Conseill%C3%A9%20en,pour%20l'adulte%20bien%20portant.
#Cette étude, publiée dans BMC Public Health, révèle que le poids moyen d'un être humain sur Terre est de 62 kg
#Poids moyenne 62kg
#besoin moyenne en proteine pour un humain 0,83g/kg/jour

besoin_proteine_jour = 0.83*62/1000
Disponibilité_intérieure_prot_kg = 3.064941e+11

#Calcul pour trouver le nombre d'humain qui pourraient etre nourris 

Disponibilité_intérieure_prot_kg / 365 / besoin_proteine_jour

Humain_nourris = 16317719840.91
Population_mond = 6997326000

print("Le nombre d'humains qui pourraient être nourris si toute la disponibilité intérieure mondiale en kg proteine de produits végétaux était utilisée pour de la nourriture: {Humain_nourris} milliards".format(Humain_nourris=Humain_nourris))

print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')


one = round(Humain_nourris / Population_mond,2)
print("Le nombre de fois qu'on pourraient nourrir la population mondiales avec la disponibilité intérieure mondiale en kg proteine de produits végétaux est de : {one} ".format(one=one))

print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

one = round(Humain_nourris / Population_mond * 100,2)
print("Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité intérieure en kg proteine de produits végétaux est de   : {one} %".format(one=one))

Le nombre d'humains qui pourraient être nourris si toute la disponibilité intérieure mondiale en kg proteine de produits végétaux était utilisée pour de la nourriture: 16317719840.91 milliards
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le nombre de fois qu'on pourraient nourrir la population mondiales avec la disponibilité intérieure mondiale en kg proteine de produits végétaux est de : 2.33 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité intérieure en kg proteine de produits végétaux est de   : 233.2 %


# Question 8 : Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [143]:
# Restriction aliments pour animaux et pertes  sur la table vegetaux 

Dispo_Anim_pertes = vegetaux[(vegetaux['Élément']=='Aliments pour animaux')
                                    |(vegetaux['Élément']=='Pertes')
                                    |(vegetaux['Élément']=='Nourriture')]


# Projections Produit', 'Élément','Année','Valeur'

Dispo_Anim_pertes = Dispo_Anim_pertes[['Produit', 'Élément','Année','Valeur']].reset_index()[['Produit','Élément','Année','Valeur']]

Dispo_Anim_pertes.head()

# Projections Produit','Ratio_kcal/kg' sur la table ratio_kcal

Ratio_kcal1= Ratio_kcal[['Produit','Ratio_kcal/kg']]

# Merge entre Dispo_Anim_pertes et Ratio_kcal1

Dispo_Anim_pertes1 = Dispo_Anim_pertes.merge(Ratio_kcal1, on='Produit', how='left')

# Création colonne Disponibilité alimentaire 

Dispo_Anim_pertes1['Disponibilité_alimentaire'] = Dispo_Anim_pertes1['Valeur']*1000000*Dispo_Anim_pertes1['Ratio_kcal/kg']
Dispo_Anim_pertes1.head()


# la disponibilité alimentaire en produits vegtetaux pour aliments pour animaux et pertes 

Dispo_Anim_pertes1[['Année','Disponibilité_alimentaire']].groupby('Année').sum().reset_index()

#Calcul pour trouver le nombre d'humains qui pourraient etre nourris (2500kcal/besoin/personne)

Humains_nourris = round(9.031517e+15/365/2500,2)

print("Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire, aliments pour animaux et pertes en calories était utilisée pour de la nourriture: {Humains_nourris} milliards".format(Humains_nourris=Humains_nourris))

print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

one = round(Humains_nourris / Population_mond *100,2)
 
print("Le pourcentage de la population mondiale qu'on pourraient nourrir en 2013 avec la disponibilité alimentaire, aliments pour animaux et pertes  en calorie  : {one} %".format(one=one))


Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire, aliments pour animaux et pertes en calories était utilisée pour de la nourriture: 9897552876.71 milliards
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le pourcentage de la population mondiale qu'on pourraient nourrir en 2013 avec la disponibilité alimentaire, aliments pour animaux et pertes  en calorie  : 141.45 %


###  Pour les proteines

In [149]:
# Merge entre table Dispo_Anim_pertes et le ratio proteine 
Dispo_Anim_pertes_prot = Dispo_Anim_pertes.merge(Ratio_prot1, on='Produit', how='left')

#Création colonne Disponibilité_alimentaire(Valeur*1000000*ratio proteine poids total)
Dispo_Anim_pertes_prot['Disponibilité_alimentaire'] = Dispo_Anim_pertes_prot['Valeur']*1000000*Dispo_Anim_pertes_prot['Ratio_protéines/poids total']
Dispo_Anim_pertes_prot[['Année','Disponibilité_alimentaire']].groupby('Année').sum().reset_index()

##https://www.cerin.org/rapports/besoin-en-proteines/#:~:text=L'Apport%20Nutritionnel%20Conseill%C3%A9%20en,pour%20l'adulte%20bien%20portant.
#Cette étude, publiée dans BMC Public Health, révèle que le poids moyen d'un être humain sur Terre est de 62 kg
#Poids moyenne 62kg
#besoin moyenne en proteine pour un humain 0,83g/kg/jour
# Calcul pour trouver le nombre d'humains qui pourraient etre nourris avec la dispo_alim

besoin_proteine_jour = 0.83*62/1000
2.032617e+11/365/besoin_proteine_jour


Humains_nourris = 10821635636.66
print("Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire, aliments pour animaux et pertes  en proteine était utilisée pour de la nourriture: {Humains_nourris} milliards".format(Humains_nourris=Humains_nourris))

print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

# Calcul pour trouver le pourcentage de la population mondiale

one = round(Humains_nourris / Population_mond *100,2)
print("Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité alimentaire, aliments pour animaux et pertes en proteine  : {one} %".format(one=one))


Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire, aliments pour animaux et pertes  en proteine était utilisée pour de la nourriture: 10821635636.66 milliards
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité alimentaire, aliments pour animaux et pertes en proteine  : 154.65 %


# Question 9 : Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [151]:
# Restriction Nourriture pour table animaux et vegetaux 
dispo_anim = animaux[animaux['Élément']=='Nourriture']
dispo_vege = vegetaux[vegetaux['Élément']=='Nourriture']

# Projections Produit','Année','Valeur' pour les table dispo_anim et dispo_vege
dispo_anim1 = dispo_anim[['Produit','Année','Valeur']].reset_index()[['Produit','Année','Valeur']]
dispo_vege1 = dispo_vege[['Produit','Année','Valeur']].reset_index()[['Produit','Année','Valeur']]

# concaténation entre dispo_anim1 et dispo_vege1
dispo_pr = pd.concat([dispo_anim1, dispo_vege1], axis=0)

# merge entre dispo_pr et Ratio_kcal1
dispo_pr1 = dispo_pr.merge(Ratio_kcal1, on='Produit', how='left')

# Creation colonne Dispo_alim 
dispo_pr1['Dispo_alim'] = dispo_pr1['Valeur']*1000000*dispo_pr1['Ratio_kcal/kg']

# Group by Année agg sum 
dispo_pr1[['Année', 'Dispo_alim']].groupby('Année').sum().reset_index()

# Calcul pour trouver le nombre d'humains qui pouuraient etre nourris (calorie)
Humains_nourris = round(7.070346e+15/365/2500,2)
print("Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire mondiale en calorie était utilisée pour de la nourriture: {Humains_nourris} milliards".format(Humains_nourris=Humains_nourris))

print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

# calcul pour le pourcentage de la population mondiale (calorie)
pourcentage = round(Humains_nourris / Population_mond *100,2)
print("Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité alimentaire mondiale en calorie  : {pourcentage} %".format(pourcentage=pourcentage))


Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire mondiale en calorie était utilisée pour de la nourriture: 7748324383.56 milliards
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité alimentaire mondiale en calorie  : 110.73 %


### pour les proteines

In [155]:
# Merge entre dispo_pr et le ratio proteine
dispo_pr2=dispo_pr.merge(Ratio_prot1, on='Produit', how='left')
dispo_pr2.head()

# Creation colonnne Dispo_alim_prot (Valeur*1000000* ratio proteine poids total)
dispo_pr2['Dispo_alim_prot']=dispo_pr2['Valeur']*1000000*dispo_pr2['Ratio_protéines/poids total']
dispo_pr2.head()

# Group by Année agg sum
dispo_pr2[['Année','Dispo_alim_prot']].groupby('Année').sum().reset_index()

humains_nourris = round(1.988936e+11/365/besoin_proteine_jour,2)
print("Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire mondiale en proteines était utilisée pour de la nourriture: {humains_nourris} milliards".format(humains_nourris=humains_nourris))
print('-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')


Pourcentage = round(humains_nourris / Population_mond *100, 2)
print("Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité alimentaire mondiale en calorie  : {Pourcentage} %".format(Pourcentage=Pourcentage))

Le nombre d'humains qui pourraient être nourris si toute la disponibilité alimentaire mondiale en proteines était utilisée pour de la nourriture: 10589078363.83 milliards
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Le pourcentage de la population mondiale qu'on pourraient nourrir avec la disponibilité alimentaire mondiale en calorie  : 151.33 %


 # Question 10 : A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [157]:
# Téléchargement fichier prévalence issu de la banque mondiale 
# Supprimer colonne Chine pour fichier sousalimentation.csv
# Reperer les valeurs uniques de la colonne description du symbole 
SousAlimentation=pd.read_csv('fr_sousalimentation.csv')
prevalence=pd.read_csv('prévalence.csv')
SousAlimentation1 = SousAlimentation[SousAlimentation['Zone']!='Chine']
SousAlimentation['Description du Symbole'].unique()

# Restriction des valeurs pour Estimation FAO
sous_nutrition = SousAlimentation1[SousAlimentation1['Description du Symbole']=='Estimation FAO']

#  Restriction des valeurs pour Estimation FAO et égale à <0.1
pays_string = sous_nutrition[sous_nutrition['Valeur']== '<0.1']

# Restriction des valeurs pour Estimation FAO différents de <0.1
pays_sous_nutrition = sous_nutrition[sous_nutrition['Valeur']!= '<0.1']

# Voir les valeurs unique et création dataframe 
pays_string1 = pays_string['Zone'].unique()
pd.DataFrame({'pays_string1':pays_string1}).head()

# projection sur colonne Country Name du dataset de la banque mondiale sur la prevalence
Preval = prevalence[['Country Name']]
Preval

# Rechercher les pays contenant le mot chine (Macao) et guyan (Guyana)
Preval[Preval["Country Name"].str.contains("Chine|Guyan", na=False)]


#Rechercher dans la colonne Country Name (banque mondiale) la liste des pays en sous_nutrtion
prev = prevalence[prevalence['Country Name'].isin(['Barbade', 'Belize', 'Brunéi Darussalam', 'Cabo Verde',
       'Région administrative spéciale de Macao, Chine', 'Chypre', 'Dominique', 'Estonie', 'Fidji',
       'Guyane', 'Îles Salomon', 'Kiribati', 'Macédoine du Nord',
       'Maldives', 'Maurice', 'Nouvelle-Calédonie', 'Polynésie française',
       'Saint-Vincent-et-les Grenadines', 'Samoa', 'Sao Tomé-et-Principe',
       'Suriname', 'Trinité-et-Tobago', 'Vanuatu'])]

prev1 = prev[['Country Name','2013 [YR2013]']].reset_index()[['Country Name', '2013 [YR2013]']].rename(columns={'2013 [YR2013]':'2013_prevalence'})
prev1

# Téléchargement dataset population (banque mondiale) 
# pour voir la population de la liste des pays en sous nutrition
POP=pd.read_csv('POP.csv')
POP1=POP[(POP['Country Name'].isin(['Barbade', 'Belize', 'Brunéi Darussalam', 'Cabo Verde',
       'Région administrative spéciale de Macao, Chine', 'Chypre', 'Dominique', 'Estonie', 'Fidji',
       'Guyane', 'Îles Salomon', 'Kiribati', 'Macédoine du Nord',
       'Maldives', 'Maurice', 'Nouvelle-Calédonie', 'Polynésie française',
       'Saint-Vincent-et-les Grenadines', 'Samoa', 'Sao Tomé-et-Principe',
       'Suriname', 'Trinité-et-Tobago', 'Vanuatu']))]

POP2 = POP1[['Country Name', '2013 [YR2013]']].reset_index()[['Country Name', '2013 [YR2013]']].rename(columns={'2013 [YR2013]':'2013_pop'})

# Merge entre dataset de la prévalence et la population (banque mondiale)
POP_sousnutrition = prev1.merge(POP2, on='Country Name', how='left')

# Conversion des colonnes en float
# création colonne Population_sous_nutrition 
# calcul prevalence*population/100
POP_sousnutrition[['2013_prevalence','2013_pop']] = POP_sousnutrition[['2013_prevalence','2013_pop']].astype(float)
POP_sousnutrition['Population_sous_nutrition']=POP_sousnutrition['2013_prevalence']*POP_sousnutrition['2013_pop']/100
POP_sousnutrition1 = POP_sousnutrition[['Country Name','Population_sous_nutrition']].rename(columns={'Country Name': 'Zone'})
POP_sousnutrition1.head()


# Restriction année 2013 pour les pays différents de <0.1
# Création colonne population_sous_nutrition (*100000)
pays_sous_nutrition1 = pays_sous_nutrition[pays_sous_nutrition['Année']=='2012-2014'][['Zone', 'Valeur']].reset_index()[['Zone', 'Valeur']]
pays_sous_nutrition1['Valeur']= pays_sous_nutrition1['Valeur'].astype(float)
pays_sous_nutrition1['Population_sous_nutrition']= pays_sous_nutrition1['Valeur']*1000000
pays_sous_nutrition2 = pays_sous_nutrition1[['Zone', 'Population_sous_nutrition']]

# Concaténation entre les pays différents de <0.1 et les pays égales à  <0.1
Proportion_sous_nutrition = pd.concat([pays_sous_nutrition2, POP_sousnutrition1], axis=0).reset_index()[['Zone', 'Population_sous_nutrition']]

# Trier les valeurs 
Proportion_sous_nutrition.sort_values(by='Population_sous_nutrition', ascending=False)

# Somme de la Population_sous_nutrition
Proportion_sous_nutrition['Population_sous_nutrition'].sum()

# Calcul pour la proportion mondiale considérée en sous_nutrition
Proportion_sous_nutrition1 = round (Proportion_sous_nutrition['Population_sous_nutrition'].sum(),2)
Pourcentage_sous_nutrition = round (Proportion_sous_nutrition1/Population_mond *100,2)

print(" La proportion de la population mondiale qui est considérée comme étant en sous-nutrition est de : {Pourcentage_sous_nutrition} %".format(Pourcentage_sous_nutrition=Pourcentage_sous_nutrition))

 

 La proportion de la population mondiale qui est considérée comme étant en sous-nutrition est de : 10.64 %


# Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.

Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal").

In [158]:
# Voir les valeurs unique pour les produits et code produits considéré comme des céréales
Cereales = pd.read_csv('fr_céréales.csv')
print(Cereales['Produit'].unique())
print(Cereales['Code Produit'].unique())

# Création dafaframe pour les produits et les codes produits 
cereales_pr= Cereales['Produit'].unique()
cereales_code= Cereales['Code Produit'].unique()
liste_pr=pd.DataFrame({'cereales_code':cereales_code,'cereales_pr':cereales_pr })
liste_pr

['Blé' 'Riz (Eq Blanchi)' 'Orge' 'Maïs' 'Millet' 'Seigle' 'Avoine'
 'Sorgho' 'Céréales, Autres']
[2511 2805 2513 2514 2517 2515 2516 2518 2520]


,cereales_code,cereales_pr
0,2511,Blé
1,2805,Riz (Eq Blanchi)
2,2513,Orge
3,2514,Maïs
4,2517,Millet
5,2515,Seigle
6,2516,Avoine
7,2518,Sorgho
8,2520,"Céréales, Autres"


# Question 11 : En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [160]:
# Restriction des produits considérés comme céréales sur la table vegetaux 
vegetaux['is_cereal']=vegetaux['Produit'].isin(liste_pr['cereales_pr'].values)
vegetaux.head(2)   

# Projection et restiction (nourriture et aliment pour animaux qui sont considérés comme des céréales)
Alim_humaine = vegetaux[(vegetaux['Élément']=='Nourriture')
                              &(vegetaux['is_cereal']==True)]

Alim_animaux = vegetaux[(vegetaux['Élément']=='Aliments pour animaux')
                              &(vegetaux['is_cereal']==True)]

# Les valeurs totales sur alimentation humaine et alimentation animale
Total_alim_humaine = Alim_humaine['Valeur'].sum()
Total_alim_animaux = Alim_animaux['Valeur'].sum()

# Calcul pour trouver proportion destinée à l'alimentation animale
propor_animale = round(Total_alim_animaux / (Total_alim_humaine+Total_alim_animaux)*100,2)
print("La proportion destinée à l'alimentation animale pour les céréales en 2013 est de  : {propor_animale} %".format(propor_animale=propor_animale))



La proportion destinée à l'alimentation animale pour les céréales en 2013 est de  : 45.91 %



# Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition.

# Repérez les 15 produits les plus exportés par ce groupe de pays.

# Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné)

# Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

# le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
# le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

# Question 12 : Donnez les 3 produits qui on t la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)


In [164]:
# Voir valeurs unique de la colonne description symbole
# restriction Estimation FAO
# Voir les pays unique en sous-nutrition
Sous_alim = pd.read_csv('fr_sousalimentation.csv')
Sous_alim['Description du Symbole'].unique()
Sous_alim_recense = Sous_alim[Sous_alim['Description du Symbole']=='Estimation FAO']
Sous_alim_recense1 = Sous_alim_recense['Zone'].unique()

# Creation Dataframe des pays en sous-nutrition
Pays_sous_nutrition1 = pd.DataFrame({'Pays_sous_nutrition': Sous_alim_recense1})
Pays_sous_nutrition1.head()


# Projections 'Zone','Élément','Produit','Valeur sur table animaux et vegetaux
# Concaténation entre animal et vegetal
animal = animaux[['Zone','Élément','Produit','Valeur']]
vegetal =vegetaux[['Zone','Élément','Produit','Valeur']]
sous_nutrition_export= pd.concat([animal, vegetal], axis=0).reset_index()[['Zone','Élément','Produit','Valeur']]

### Repérez les 15 produits les plus exportés par ce groupe de pays.

In [165]:
# Restriction des pays en sous nutrition sur la table sous-nutrition-export  
sous_nutrition_export1 = sous_nutrition_export[(sous_nutrition_export['Zone'].isin(Pays_sous_nutrition1['Pays_sous_nutrition'].values))]

# restriction Exportations - Quantité 
# Group by produit agg sum et tri des produits décroissants
pr_import = sous_nutrition_export1[sous_nutrition_export1['Élément']=='Exportations - Quantité'].groupby('Produit').sum().reset_index().sort_values(by='Valeur', ascending=False).head(15)
pr_import


,Produit,Valeur
52,Maïs,54722.0
39,Huile de Palme,46349.0
51,Manioc,35851.0
12,Blé,33756.0
78,Riz (Eq Blanchi),33122.0
82,Sucre Eq Brut,26326.0
49,"Légumes, Autres",20281.0
8,Bananes,17996.0
48,Lait - Excl Beurre,15906.0
80,Soja,15884.0


# Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné)¶

In [166]:
# Concaténation entre table animal et vegetal 
anim_vege_import = pd.concat([animal, vegetal], axis=0).reset_index()[['Zone','Élément','Produit','Valeur']]
anim_vege_import.head()

# restriction des produits  exportés par les pays en sous nutrition
Plus_import_pr = anim_vege_import[(anim_vege_import['Produit'].isin(pr_import['Produit'].values))]

# restriction Importations - Quantité' et tri par ordre décroissant
gb_pr = Plus_import_pr[Plus_import_pr['Élément']=='Importations - Quantité']
gb_pr.sort_values(by='Valeur', ascending=False).head(200)

,Zone,Élément,Produit,Valeur
58355,"Chine, continentale",Importations - Quantité,Soja,63381.0
58203,"Chine, continentale",Importations - Quantité,Manioc,29046.0
88015,Japon,Importations - Quantité,Maïs,14403.0
41025,Arabie saoudite,Importations - Quantité,Orge,10547.0
65588,Égypte,Importations - Quantité,Blé,10331.0
...,...,...,...,...
87134,Italie,Importations - Quantité,"Légumes, Autres",1061.0
36191,Viet Nam,Importations - Quantité,Lait - Excl Beurre,1041.0
114915,Pologne,Importations - Quantité,Blé,1037.0
55595,Canada,Importations - Quantité,Maïs,1035.0


# Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. 

In [170]:
# Groupement par produits des plus grandes importation des produits exportés par les pays en sous nutrition 
gb_pr_import = gb_pr.groupby('Produit').sum().reset_index()
gb_pr_import.sort_values(by='Valeur', ascending=False).reset_index()[['Produit', 'Valeur']]

,Produit,Valeur
0,Blé,185377.0
1,Maïs,122691.0
2,Lait - Excl Beurre,116806.0
3,Soja,102082.0
4,Sucre Eq Brut,61615.0
5,Huile de Palme,48858.0
6,"Légumes, Autres",43333.0
7,Orge,38689.0
8,Manioc,38370.0
9,Riz (Eq Blanchi),37618.0


# Ensuite, calculez pour chaque produit les 2 quantités suivantes :
# le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.


In [184]:
# restriction des produits les plus importés
# Restriction Autres utilisations (non alimentaire)'
# Restriction Disponibilité intérieure
pr_IMPORT = anim_vege_import[anim_vege_import['Produit'].isin(gb_pr_import['Produit'].values)]

Autres_util = pr_IMPORT[pr_IMPORT['Élément']=='Autres utilisations (non alimentaire)']

disp_int = pr_IMPORT[pr_IMPORT['Élément']=='Disponibilité intérieure']

# Groupement par produits pour Autres utilisations (non alimentaire) et Disponibilité intérieure
# Renommer les colonnes
Autres_util1 = Autres_util.groupby('Produit').sum().reset_index().rename(columns={'Valeur':'Autres utilisations'})
disp_int1 = disp_int.groupby('Produit').sum().reset_index().rename(columns={'Valeur':'disponibilité intérieure'})

# Merge entre Autres_util1 et disp_int1
# Création colonne ratio(autres utilisation/dispo_int)
Ratio1 = Autres_util1.merge(disp_int1, on='Produit', how='left')
Ratio1['ratio(autres utilisation/dispo_int)']=Ratio1['Autres utilisations']/Ratio1['disponibilité intérieure']
Ratio1.sort_values(by='ratio(autres utilisation/dispo_int)', ascending=False)[['Produit', 'ratio(autres utilisation/dispo_int)']].head(3)

,Produit,ratio(autres utilisation/dispo_int)
3,Huile de Palme,0.698109
7,Maïs,0.198144
6,Manioc,0.140211


# le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)¶

In [182]:
# Restriction pour alimentation animale et humaine 
Alim_animaux = pr_IMPORT[pr_IMPORT['Élément']=='Aliments pour animaux']
Alim_anim_hum = pr_IMPORT[(pr_IMPORT['Élément']=='Aliments pour animaux')
                         |(pr_IMPORT['Élément']=='Nourriture')]

# Groupement par produits pour l'alimentation animale et humaine 
Alim_animaux1 = Alim_animaux.groupby('Produit').sum().reset_index().rename(columns={'Valeur': 'nourriture animale'})
Alim_anim_hum1 = Alim_anim_hum.groupby('Produit').sum().reset_index().rename(columns={'Valeur': 'nourriture (animale + humaine)'})

# Merge entre Alim_animaux1 et Alim_anim_hum1
# Création colonne ratio_nourri_anim/nourri_anim_hum
Ratio2 = Alim_animaux1.merge(Alim_anim_hum1, on='Produit', how='left')
Ratio2['ratio_nourri_anim/nourri_anim_hum']= Ratio2['nourriture animale']/Ratio2['nourriture (animale + humaine)']
Ratio2.sort_values(by='ratio_nourri_anim/nourri_anim_hum', ascending=False)[['Produit', 'ratio_nourri_anim/nourri_anim_hum']].head(3)

,Produit,ratio_nourri_anim/nourri_anim_hum
8,Orge,0.931686
7,Maïs,0.813520
12,Soja,0.621391


# Question 13 : Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [187]:
# Projection zone, element et restriction États-Unis d'Amérique, Production
product_anim_usa = animaux[(animaux['Zone']=="États-Unis d'Amérique")
                                  &(animaux['Élément']=='Production')]

# Chercher liste produits dans table product_anim_usa['Produit'] 
product_anim_usa1 = product_anim_usa[(product_anim_usa['Produit'].isin(['Viande de Bovins','Viande de Suides','Viande de Volailles']))]

diminution = product_anim_usa1[['Produit', 'Valeur']].reset_index()[['Produit', 'Valeur']]


# Appliquer -10% sur la colonne Valeur
diminution['-10%production_tonnes'] = diminution['Valeur'].apply(lambda x : (x*1000)*10/100 )
diminution

#7 kg de céréales sont nécessaires pour produire un kilo de bœuf, 
    #4 pour 1 kg de porc, 2 pour 1 kg de poulet. 

#bovin = 7000/1000
#print(bovin)
#porc= 4000/1000
#print(porc)
#poulet= 2000/1000
#print(poulet)


#Création colonne consomation animale pour 1 tonne animale
# Création colonne libération tonnes céréales
diminution['Consommation_céréale(tonne)_pour_1tonne_animale'] = [7, 4, 2]
diminution['Libération_tonnes_céréales']= diminution['-10%production_tonnes']*diminution['Consommation_céréale(tonne)_pour_1tonne_animale']
diminution

# Total de la libération_tonnes_céréales
liberation_cereales = diminution['Libération_tonnes_céréales'].sum()
print("Si les USA diminué leur production de 10% de produits animaux, on pourraient libérer: {liberation_cereales} tonnes de céréales".format(liberation_cereales=liberation_cereales))




Si les USA diminué leur production de 10% de produits animaux, on pourraient libérer: 16430300.0 tonnes de céréales


# Question 14 : En Thaïlande, quelle proportion de manioc est exportée ? 

In [188]:
# Projections et restrictions 
export = vegetaux[(vegetaux['Zone']=='Thaïlande')
                &(vegetaux['Élément']=='Exportations - Quantité')
                &(vegetaux['Produit']=='Manioc')]


prod = vegetaux[(vegetaux['Zone']=='Thaïlande')
                &(vegetaux['Élément']=='Production')
                &(vegetaux['Produit']=='Manioc')]

# Projections
prod1 = prod[['Zone','Valeur']]
export1 = export[['Zone', 'Valeur' ]]

# Merge entre prod1 et export1
propor_manioc = export1.merge(prod1, on='Zone', how='left').rename(columns={'Valeur_x':'Exportation', 'Valeur_y':'Production'})
propor_manioc

# Création colonne proportion_manioc_export
propor_manioc['Propor_manioc_export'] = propor_manioc['Exportation']/propor_manioc['Production']*100
propor_manioc

propor_manioc = round(83.41273,2)

print("La proportion de manioc exportée en Thaïlande en 2013 est de : {propor_manioc} %".format(propor_manioc=propor_manioc))

La proportion de manioc exportée en Thaïlande en 2013 est de : 83.41 %


# Quelle est la proportion de personnes en sous-nutrition?

In [189]:
#Projection(Zone) et restriction(Thaïlande) dataframe Proportion_sous_nutrition
sous_nutrition_thai = Proportion_sous_nutrition[Proportion_sous_nutrition['Zone']=='Thaïlande'].reset_index()[['Zone','Population_sous_nutrition']]
sous_nutrition_thai

#Projection(Zone) et restriction(Thaïlande) table population
pop_thai = populations[populations['Zone']=='Thaïlande'].reset_index()[['Zone','Population']]
pop_thai

#Merge entre sous_nutrition_thai et pop_thai
#Création colonne Proportion_sous_nutrition
propor_thai_pop = sous_nutrition_thai.merge(pop_thai, on='Zone', how='left')
propor_thai_pop['Proportion_sous_nutrition'] = propor_thai_pop['Population_sous_nutrition']/(propor_thai_pop['Population']*1000)*100
propor_thai_pop


propor_sous_nutri_thai = round(8.356837,2)

print("La proportion de personnes en sous nutrition en Thaïlande est de : {propor_sous_nutri_thai} %".format(propor_sous_nutri_thai=propor_sous_nutri_thai))


La proportion de personnes en sous nutrition en Thaïlande est de : 8.36 %
